Trouxemos o Label Binarizer para regularizar as classes do nosso modelo.
Importamos o Sequential, que é a base da construção de camadas.
Importamos o Dense, que é o método de construção de camadas totalmente conectadas.
Importamos o SGD (Stochastic Gradient Descent), que é o otimizador de gradiente descendente tradicional.

Além desses métodos essenciais, cabe ressaltar que:


Importamos o classification report, método que nos mostrará o resultado de nossa demonstração em detalhes (muito recomendado para o uso no dia a dia com problemas de classificação de um modo geral).
Importamos o MNIST, que é o nosso conjunto de dados do qual falaremos mais adiante.
Importamos o pyplot para visualizarmos a evolução do aprendizado no nosso modelo, bem como o numpy para regularizar o dimensionamento dos nossos dados.

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np

nossa rede neural vai aprender a reconhecer o padrão de escrita de números. Para conseguirmos esse conjunto

In [4]:
print('[INFO] accessing MNIST...')
((trainX, trainY), (testX, testY)) = mnist.load_data()

[INFO] accessing MNIST...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


normalizaremos os dados para que fiquem entre 0 e 1, e faremos isso dividindo o conjunto por 255 (valor máximo de um pixel)

In [5]:
trainX = trainX.reshape((trainX.shape[0], 28 * 28 * 1))
testX = testX.reshape((testX.shape[0], 28 * 28 * 1))
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0

para adequar a última camada, a de saída, vamos binarizar a classe

In [6]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

vamos definir a arquitetura da nossa rede neural

In [7]:
model = Sequential()
model.add(Dense(256, input_shape=(784,), activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Uma camada de entrada de 784 nós, um para cada pixel da imagem em questão, que se conectará a uma camada oculta densa de 256 nós pela função de ativação da sigmoide.
Depois, a primeira camada oculta se conectará à segunda, de 128 nós, também por sigmoide.
Esta se conectará à última camada de predição com 10 nós conectados a partir da Softmax. São 10 nós, porque temos 10 possíveis dígitos.

Para treinar o modelo, vamos usar como otimizador do gradiente o SGD, aquele baseado no gradiente descendente, e com taxa de aprendizado 0.01. Também faremos uso da métrica de acurácia para acompanhar o modelo.


Para calcular a perda ou função de custo, vamos usar a entropia cruzada categórica (categorical_crossentropy), que é a mais utilizada em problemas de classificação.

In [8]:
sgd = SGD(0.01)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
H = model.fit(trainX, trainY, validation_data=(testX, testY),epochs=100, batch_size=128)

Epoch 1/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.1435 - loss: 2.3067 - val_accuracy: 0.3839 - val_loss: 2.2378
Epoch 2/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.3718 - loss: 2.2230 - val_accuracy: 0.5313 - val_loss: 2.1653
Epoch 3/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5244 - loss: 2.1454 - val_accuracy: 0.5645 - val_loss: 2.0643
Epoch 4/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6189 - loss: 2.0351 - val_accuracy: 0.6634 - val_loss: 1.9136
Epoch 5/100


KeyboardInterrupt: 

entenda que para as épocas da nossa rede, vamos escolher 100 épocas, ou seja, a rede tem 100 iterações para convergir e apreender, e vamos apresentar lotes de 128 imagens cada por iteração.

In [9]:
predictions = model.predict(testX, batch_size=128)
print(classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=[str(x) for x in lb.classes_]))

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
              precision    recall  f1-score   support

           0       0.60      0.97      0.74       980
           1       0.42      1.00      0.59      1135
           2       0.85      0.63      0.72      1032
           3       0.74      0.62      0.67      1010
           4       0.85      0.70      0.77       982
           5       0.00      0.00      0.00       892
           6       0.91      0.80      0.85       958
           7       0.66      0.81      0.73      1028
           8       0.89      0.50      0.64       974
           9       0.78      0.50      0.61      1009

    accuracy                           0.66     10000
   macro avg       0.67      0.65      0.63     10000
weighted avg       0.67      0.66      0.64     10000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


vamos utilizar a classification_report, uma função do sklearn que compara os valores preditos com os reais, passados como argumentos.

In [10]:
plt.style.use('ggplot')
plt.figure()
plt.plot(np.arange(0, 100), H.history['loss'], label='train_loss')
plt.plot(np.arange(0, 100), H.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, 100), H.history['accuracy'], label='train_acc')
plt.plot(np.arange(0, 100), H.history['val_accuracy'], label='val_acc')
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend()

NameError: name 'H' is not defined

<Figure size 640x480 with 0 Axes>